<a href="https://colab.research.google.com/github/LarsBryld/socialdataanalysis2020/blob/master/Oxford.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

ox = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv" , parse_dates=["Date"])

covid = pd.read_csv("https://open-covid-19.github.io/data/data.csv", parse_dates=["Date"], na_values='-')



#Define focus contries based on stock indecies
focuscountries = set(['China', 'Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Italy', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Netherlands', 'Poland', 'Portugal', 'Spain',
                  'Sweden', 'Switzerland', 'United Kingdom', 'Canada', 'United States', 'Australia', 'New Zealand', 'Hong Kong', 'Japan', 'South Korea', 'Singapore',
                  'Taiwan', 'Brazil', 'Israel', 'India', 'Mexico', 'South Africa', 'Russia'])


#Create dataframes of only focus countries
ox = ox[ox['CountryName'].isin(focuscountries)]
covid = covid[covid['CountryName'].isin(focuscountries)]


#Sometimes, due to data entry errors, we have isgeneral=1 when the underlying indicator is 0 or null. These cases should be disregarded. Only use isgeneral=1 values when the underlying indicator is 1 or higher.

#ox = ox[(ox['S1_IsGeneral'] != 1) & (ox['S1_School closing'] > 0)]
#ox =  (ox['S2_IsGeneral'] != 1) & (ox['S2_Workplace closing'] > 0) 
#ox = ox[(ox['S2_IsGeneral'] != 1) & (ox['S2_Workplace closing'] > 0)]
#ox = ox[(ox['S3_IsGeneral'] != 1) & (ox['S3_Cancel public events'] > 0)]
#ox = ox[(ox['S4_IsGeneral'] != 1) & (ox['S4_Close public transport'] > 0)]
#ox = ox[(ox['S5_IsGeneral'] != 1) & (ox['S5_Public information campaigns'] > 0)]
#ox = ox[(ox['S6_IsGeneral'] != 1) & (ox['S6_Restrictions on internal movement'] > 0)]
#drop2 = ox[(ox['S1_IsGeneral'] == 1) & (ox['S1_School closing'] == 0)] 


#Nine of the indicators (S1–S7, S12 and S13) take policies such as school closures, travel bans, etc, and are recorded on an ordinal scale
#policies = ox[['CountryName', 'Date', 'S1_School closing', 'S2_Workplace closing', 'S3_Cancel public events', 'S3_Cancel public events', 'S4_Close public transport', 'S5_Public information campaigns', 'S6_Restrictions on internal movement', 'S7_International travel controls']]

#the others (S8–S11) are financial indicators such as fiscal or monetary measures
#financial = ox[['CountryName', 'Date', 'S8_Fiscal measures', 'S9_Monetary measures', 'S10_Emergency investment in health care', 'S11_Investment in Vaccines']]

#The tracker aggregates the policy scores (not the financial scores) into a common ‘Stringency Index’.
#stringency = ox[['CountryName', 'Date', 'StringencyIndex']]

ox_country = (ox['CountryName']).unique()
covid_country = (covid['CountryName']).unique()



In [0]:
#Renaming the NaN regional codes to 'Country', since we then can isolate total country level data using this variable

covid['RegionCode'] = covid['RegionCode'].fillna('Country')


In [171]:

print("The Covid data has {} unique countries and {} unique dates".format(len(covid_country),len(covid['Date'].unique())))
print("The Covid data has start data {} and end date {} ".format(min(covid['Date']),max(covid['Date'])))
print("The Oxford data has {} unique countries and {} unique dates".format(len(ox_country),len(ox['Date'].unique())))
print("The Oxford data has start data {} and end date {} ".format(min(ox['Date']),max(ox['Date'])))
print("The country in Oxford data that is NOT in Covid data is THIS Function IS MISSING {:10.0f} ".format(len(ox['CountryName'].unique()),len(ox['Date'].unique())))

# https://www.dataquest.io/blog/python-datetime-tutorial/ useful link to corect the printed date format


The Covid data has 31 unique countries and 122 unique dates
The Covid data has start data 2019-12-30 00:00:00 and end date 2020-04-29 00:00:00 
The Oxford data has 33 unique countries and 121 unique dates
The Oxford data has start data 2020-01-01 00:00:00 and end date 2020-04-30 00:00:00 
The country in Oxford data that is NOT in Covid data is THIS Function IS MISSING         33 


In [0]:
#Creating single country example for test of merging. 
ox_den = ox[ox['CountryName'] == 'Denmark']
covid_den = covid[covid['CountryName'] == 'Denmark']

#Creating single country example for merging test.
ox_it = ox[ox['CountryCode'] == 'ITA']
covid_it = covid[(covid['CountryName'] == 'Italy') & (covid['Key'] == 'IT')]


In [0]:
#reduce the variables that are not being used both in oxford and covid data
drop_ox = ['CountryName', 'CountryCode'] 
ox_it = ox_it.drop(drop_ox, axis=1)

drop_covid = ['Key', 'CountryCode'] 
covid_it = covid_it.drop(drop_covid, axis=1)

In [177]:
# Summing total confirmed cases for it to compare against the entire data set 
it = pd.merge(covid_it,ox_it, how='inner', on='Date')
sum(it['Confirmed'])


5733120.0

In [179]:
# Same as above but for DK 
den = pd.merge(covid_den,ox_den, how='inner', on='Date')
sum(den['Confirmed'])

201835.0

In [0]:
#making a general solution to the above mereging example. oxford_data is the complete data set 

oxford_date = pd.merge(covid,ox, how='inner', on=['Date', 'CountryName'])


In [184]:
#testing that the manual merging example creates the same results as in the oxford_data

it_test = oxford_date[(oxford_date['CountryName'] == 'Italy') & (oxford_date['RegionCode'] == 'Country')]
sum(it_test['Confirmed'])

5733120.0

In [182]:
#same as above but for denmark

dk_test = oxford_date[(oxford_date['CountryName'] == 'Denmark') & (oxford_date['RegionCode'] == 'Country')]
sum(dk_test['Confirmed'])

201835.0

In [0]:

#plotting statement

string = den['StringencyIndex']
cases = den['ConfirmedCases']       

string.plot.line()
#cases.plot.line()

A full description of the strengency index can be found here: 

https://www.bsg.ox.ac.uk/sites/default/files/Calculation%20and%20presentation%20of%20the%20Stringency%20Index.pdf

Additional desciption on a less detailed level: 

https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker



equals(self, other)

Test whether two objects contain the same elements.

https://pandas.pydata.org/docs/getting_started/basics.html


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

